In [1]:
"""
This notebook is used to interactively view images in a folder to check them for issues
"""

'\nThis notebook is used to interactively view images in a folder to check them for issues\n'

In [1]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from copy import deepcopy

from scipy import ndimage
from skimage.measure import label

from numpy import linalg
from scipy.sparse.linalg import eigs

from skimage.exposure import match_histograms

from utils import is_wing_facing_up, segment_contour

In [5]:
seen = []
problem_wings = []
FOLDER = '../2_live_bees/6_cropped_and_flipped/'
subset = []



In [7]:

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import os
from PIL import Image, ImageFilter

def imageEditor(folder, filename, blur, blurRadius): 
	# Opens a image in RGB mode
	im = Image.open('{}/{}'.format(folder, filename))

	if(blur == True):
		# blur only if checkbox checked
		im = im.filter(ImageFilter.GaussianBlur(blurRadius))
		
	# show the image
	plt.figure(figsize=(9,9))
	plt.imshow(im)
	plt.axis('off')
	plt.title(filename)
	print(filename)
	plt.show()

# creating the widgets
folderWidget = widgets.Text(FOLDER)
fns = [x for x in sorted(os.listdir(folderWidget.value)) if x not in seen]
if len(subset):
    fns = [x for x in fns if x in subset]
fileWidget = widgets.Dropdown(options = fns)
# update function to update fileslist when folder name changes
def updateFilesList(*args):
	filesList = [file for file in os.listdir(folderWidget.value) if file.endswith(('.png', '.jpg', '.jpeg','.gif'))]
	fileWidget.options = filesList
	fileWidget.value = filesList[0]

# observer folder widget and call updateFilesList whenever its value changes
folderWidget.observe(updateFilesList, 'value')


button_prev = widgets.Button(description="Previous!")
button_next = widgets.Button(description="Next!")
button_add = widgets.Button(description="Add to list")
button_remove = widgets.Button(description="Remove from list")

display(button_prev)
display(button_next)
display(button_add)
display(button_remove)
interact(
	imageEditor,
	filename = fileWidget,
	folder = folderWidget,
	blur = widgets.Checkbox(value = False),
	blurRadius = widgets.FloatSlider(min = 0, max = 5, description = "Blur Radius")
)
output = widgets.Output()
display(output)

def on_button_prev_clicked(a):
    with output:
        cur = fileWidget.get_state()['index']
        n_fps = len(fileWidget.get_state()['_options_labels'])
        if cur > 0:
            fileWidget.set_state({'index':cur - 1})
            print(f"{cur} / {n_fps}", end='\r')

def on_button_next_clicked(b):
    global seen
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        n_fps = len(fileWidget.get_state()['_options_labels'])
        fileWidget.set_state({'index':cur + 1})
        print(f"{cur} / {n_fps}                             ", end='\r')

    if fn not in seen:
        seen += [fn]
    

def on_button_add_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Added {fn}", end='\r')

    if fn not in problem_wings:
        problem_wings += [fn]

def on_button_remove_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Removed {fn}", end='\r')
    
    problem_wings.remove(fn)

button_prev.on_click(on_button_prev_clicked)
button_next.on_click(on_button_next_clicked)
button_add.on_click(on_button_add_clicked)
button_remove.on_click(on_button_remove_clicked)

Button(description='Previous!', style=ButtonStyle())

Button(description='Next!', style=ButtonStyle())

Button(description='Add to list', style=ButtonStyle())

Button(description='Remove from list', style=ButtonStyle())

interactive(children=(Text(value='../2_live_bees/6_cropped_and_flipped/', description='folder'), Dropdown(desc…

Output()

In [10]:
df = []
for fn in seen:
    if fn in problem_wings:
        df += [[fn, True]]
    else:
        df += [[fn, False]]

df = pd.DataFrame(df, columns=['Filename','Damaged?'])
#df.to_csv('live_bees_damaged.csv')

In [4]:
set(problem_wings)

{'Hive01_Sheet_02_slideww43_left.png',
 'Hive01_Sheet_02_slideww43_right.png',
 'Hive01_Sheet_03_slideww21-x_left.png',
 'Hive02_Sheet_01_slide23_left.png',
 'Hive02_Sheet_01_slide45_right.png',
 'Hive02_Sheet_02_slide103_left.png',
 'Hive02_Sheet_02_slide105_right.png',
 'Hive02_Sheet_03_slideww47_left.png',
 'Hive02_Sheet_03_slideww70_left.png',
 'Hive03_Sheet_01_slide115_left.png',
 'Hive03_Sheet_01_slideww45_right.png',
 'Hive04_Sheet_01_slide38_right.png',
 'Hive04_Sheet_01_slideww28_left.png',
 'Hive04_Sheet_02_slide38_right.png',
 'Hive04_Sheet_02_slideww28_left.png',
 'Hive05_Sheet_01_slideww82_left.png',
 'Hive06_Sheet_02_slide62_right.png',
 'Hive06_Sheet_03_slide115_left.png',
 'Hive06_Sheet_03_slide80_left.png',
 'Hive06_Sheet_04_slide125_right.png',
 'Hive07_Sheet_01_slide33_left.png',
 'Hive07_Sheet_04_slide120_left.png',
 'Hive07_Sheet_04_slide122_left.png',
 'Hive07_Sheet_04_slide129_left.png',
 'Hive07_Sheet_05_slide160_left.png',
 'Hive07_Sheet_05_slide162_left.png',


In [6]:
set(seen) - set(problem_wings)

{'2024_06_05_h01bee01.png',
 '2024_06_05_h01bee16.png',
 '2024_06_05_h01bee19.png',
 '2024_06_05_h01bee21.png',
 '2024_06_05_h01bee23.png',
 '2024_06_05_h01bee27.png',
 '2024_06_05_h01bee29.png',
 '2024_06_05_h01bee36.png',
 '2024_06_05_h01bee37.png'}

In [5]:
problem_wings = ['2024_06_05_h01bee01.png',
 '2024_06_05_h01bee19.png',
 '2024_06_05_h01bee21.png',
 '2024_06_05_h01bee27.png',
 '2024_06_05_h01bee32.png',
 '2024_06_05_h01bee36.png',
 '2024_06_05_h01bee45.png',
 '2024_06_06_h03bee39.png',
 '2024_06_06_h05bee07.png',
 '2024_06_06_h05bee11.png',
 '2024_06_06_h05bee32.png',
 '2024_06_07_h02bee03.png',
 '2024_06_07_h02bee34.png',
 '2024_06_07_h02bee38.png',
 '2024_06_10_h02b01.png',
 '2024_06_10_h02b32.png',
 '2024_06_10_h02b36.png',
 '2024_06_10_h02b42.png',
 '2024_06_11_h01b41.png',
 '2024_06_25_h03b10.png',
 '2024_06_27_h01b41.png',
 '2024_07_04_h24b02.png',
 '2024_07_10_h24b02.png',
 '2024_07_17_h31b48.png',
 '2024_07_18_h24b02.png',
 '2024_07_23_h21b28.png',
 '2024_07_25_h24b02.png',
 '2024_07_31_h31b74.png',
 '2024_08_05_h32b11.png'] # these are the ones to remove from the segmentations and cell size calculations. 

['2024_06_05_h01bee01.png',
 '2024_06_05_h01bee19.png',
 '2024_06_05_h01bee21.png',
 '2024_06_05_h01bee27.png',
 '2024_06_05_h01bee32.png',
 '2024_06_05_h01bee36.png',
 '2024_06_05_h01bee45.png',
 '2024_06_06_h03bee39.png',
 '2024_06_06_h05bee07.png',
 '2024_06_06_h05bee11.png',
 '2024_06_06_h05bee32.png',
 '2024_06_07_h02bee03.png',
 '2024_06_07_h02bee34.png',
 '2024_06_07_h02bee38.png',
 '2024_06_10_h02b01.png',
 '2024_06_10_h02b32.png',
 '2024_06_10_h02b36.png',
 '2024_06_10_h02b42.png',
 '2024_06_11_h01b41.png',
 '2024_06_25_h03b10.png',
 '2024_06_27_h01b41.png',
 '2024_07_04_h24b02.png',
 '2024_07_10_h24b02.png',
 '2024_07_17_h31b48.png',
 '2024_07_18_h24b02.png',
 '2024_07_23_h21b28.png',
 '2024_07_25_h24b02.png',
 '2024_07_31_h31b74.png',
 '2024_08_05_h32b11.png']

In [23]:
# Used to go through sets of images and compare them, ie individual wings and the original slides to 
# do some checking that they matched.

seen = []
problem_wings = []
FOLDER = '../1_slides/1_slides/'
FOLDER2 = '../1_slides/2_individual_wings/'
subset = []

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import os
from PIL import Image, ImageFilter

def imageEditor(folder, folder2, filename, blur, blurRadius): 
    # Opens a image in RGB mode
    im = Image.open('{}/{}'.format(folder, filename))
    im2 = Image.open('{}/{}'.format(folder2, filename.replace('.png','_left.png')))
    im3 = Image.open('{}/{}'.format(folder2, filename.replace('.png','_right.png')))
    
    if(blur == True):
        # blur only if checkbox checked
        im = im.filter(ImageFilter.GaussianBlur(blurRadius))
        
    # show the image
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.axis('off')
    plt.title(filename)
    plt.show()
    try:
        # show the image2
    	plt.figure(figsize=(5,5))
    	plt.imshow(im2)
    	plt.axis('off')
    	plt.title(filename)
    	print(filename)
    	plt.show()
    	# show the image3
    	plt.figure(figsize=(5,5))
    	plt.imshow(im3)
    	plt.axis('off')
    	plt.title(filename)
    	print(filename)
    	plt.show()
    except:
        pass

# creating the widgets
folderWidget = widgets.Text(FOLDER)
fns = [x for x in sorted(os.listdir(folderWidget.value)) if x not in seen]
if len(subset):
    fns = [x for x in fns if x in subset]
fileWidget = widgets.Dropdown(options = fns)
# update function to update fileslist when folder name changes
def updateFilesList(*args):
	filesList = [file for file in os.listdir(folderWidget.value) if file.endswith(('.png', '.jpg', '.jpeg','.gif'))]
	fileWidget.options = filesList
	fileWidget.value = filesList[0]

# observer folder widget and call updateFilesList whenever its value changes
folderWidget.observe(updateFilesList, 'value')


button_prev = widgets.Button(description="Previous!")
button_next = widgets.Button(description="Next!")
button_add = widgets.Button(description="Add to list")
button_remove = widgets.Button(description="Remove from list")

display(button_prev)
display(button_next)
display(button_add)
display(button_remove)
interact(
	imageEditor,
	filename = fileWidget,
	folder = folderWidget,
    folder2 = FOLDER2,
	blur = widgets.Checkbox(value = False),
	blurRadius = widgets.FloatSlider(min = 0, max = 5, description = "Blur Radius")
)
output = widgets.Output()
display(output)

def on_button_prev_clicked(a):
    with output:
        cur = fileWidget.get_state()['index']
        n_fps = len(fileWidget.get_state()['_options_labels'])
        if cur > 0:
            fileWidget.set_state({'index':cur - 1})
            print(f"{cur} / {n_fps}", end='\r')

def on_button_next_clicked(b):
    global seen
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        n_fps = len(fileWidget.get_state()['_options_labels'])
        fileWidget.set_state({'index':cur + 1})
        print(f"{cur} / {n_fps}                             ", end='\r')

    if fn not in seen:
        seen += [fn]
    

def on_button_add_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Added {fn}", end='\r')

    if fn not in problem_wings:
        problem_wings += [fn]

def on_button_remove_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Removed {fn}", end='\r')
    
    problem_wings.remove(fn)

button_prev.on_click(on_button_prev_clicked)
button_next.on_click(on_button_next_clicked)
button_add.on_click(on_button_add_clicked)
button_remove.on_click(on_button_remove_clicked)

Button(description='Previous!', style=ButtonStyle())

Button(description='Next!', style=ButtonStyle())

Button(description='Add to list', style=ButtonStyle())

Button(description='Remove from list', style=ButtonStyle())

interactive(children=(Text(value='../1_slides/1_slides/', description='folder'), Text(value='../1_slides/2_ind…

Output()